In [12]:
import requests
import pandas as pd

## time series
---
### Using the code from the lesson as a guide and the REST API from https://python.zgulde.net/api/v1/items as we did in the lesson, create a dataframe named items that has all of the data for items.

In [13]:
base_url = 'https://python.zgulde.net'
response = requests.get('https://python.zgulde.net/api/v1/items')

data = response.json()

items = pd.DataFrame(data['payload']['items'])
items.head()


,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [14]:
response = requests.get(base_url + data['payload']['next_page'])
data = response.json()

current_page = data['payload']['page']
max_page = data['payload']['max_page']
next_page = data['payload']['next_page']

print(f'current_page: {current_page}')
print(f'max_page: {max_page}')
print(f'next_page: {next_page}')

items = pd.concat([items, pd.DataFrame(data['payload']['items'])]).reset_index()

current_page: 2
max_page: 3
next_page: /api/v1/items?page=3


In [15]:
response = requests.get(base_url + data['payload']['next_page'])
data = response.json()

current_page = data['payload']['page']
max_page = data['payload']['max_page']
next_page = data['payload']['next_page']

print(f'current_page: {current_page}')
print(f'max_page: {max_page}')
print(f'next_page: {next_page}')

items = pd.concat([items, pd.DataFrame(data['payload']['items'])]).reset_index()

current_page: 3
max_page: 3
next_page: None


In [16]:
items.shape

(50, 8)

### Do the same thing, but for stores (https://python.zgulde.net/api/v1/stores)

In [17]:
base_url = 'https://python.zgulde.net'
response = requests.get('https://python.zgulde.net/api/v1/stores')

data = response.json()

stores = pd.DataFrame(data['payload']['stores'])
stores.head()

current_page = data['payload']['page']
max_page = data['payload']['max_page']
next_page = data['payload']['next_page']

print(f'current_page: {current_page}')
print(f'max_page: {max_page}')
print(f'next_page: {next_page}')


current_page: 1
max_page: 1
next_page: None


### Extract the data for sales (https://python.zgulde.net/api/v1/sales). There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.

In [18]:
response = requests.get('https://python.zgulde.net/api/v1/sales')

data = response.json()

next_page = data['payload']['next_page']
max_page = data['payload']['max_page']

print(max_page)


183


In [19]:
response_sales = requests.get('https://python.zgulde.net/api/v1/sales')

data_sales = response_sales.json()

data_sales.keys()

max_page_sales = data_sales['payload']['max_page']

sales_url = 'https://python.zgulde.net/api/v1/sales?page='

sales = pd.DataFrame(data_sales['payload']['sales'])

for i in range(2, max_page_sales + 1):
    response = requests.get(sales_url + str(i))
    json = response.json()
    df = pd.DataFrame(json['payload']['sales'])
    sales = pd.concat([sales, df])


In [20]:
sales.reset_index(inplace=True)

Save the data in your files to local csv files so that it will be faster to access in the future.

In [21]:
items.to_csv('items.csv')
stores.to_csv('stores.csv')
sales.to_csv('sales.csv')

Combine the data from your three separate dataframes into one large dataframe.

In [22]:

super_df = sales.merge(items, how='left',
                       left_on='item', right_on='item_id')

super_df = super_df.merge(stores, how='left',
                          left_on='store', right_on='store_id')
super_df.drop(columns=['index_x', 'item', 'sale_id', 'store', 'level_0',
                       'index_y', 'item_id', 'item_upc12', 'item_upc14',
                       'store_id'], inplace=True)

In [23]:
super_df.to_csv('merged_df.csv')

Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here: https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv

In [24]:
power_df = pd.read_csv('https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv')
power_df.to_csv('power.csv')


Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions in the acquire.py file and be able to re-run the functions and get the same data.

In [25]:
def get_items():
    
    base_url = 'https://python.zgulde.net'
    
    response = requests.get('https://python.zgulde.net/api/v1/items')
    
    data = response.json()
    
    items = pd.DataFrame(data['payload']['items'])
    
    next_page = data['payload']['next_page']
    
    while next_page != None:
        
        response = requests.get(base_url + next_page)
        
        data = response.json()
        
        next_page = data['payload']['next_page']
        
        new_items = pd.DataFrame(data['payload']['items'])
        
        items = pd.concat([items, new_items])
    
    items.reset_index(drop=True, inplace=True)
    
    #items.to_csv('items.csv')
    
    return items